# Merging data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read data 

# opioid
opioid = pd.read_parquet("/Users/preetkhowaja/Documents/MIDS Fall 2021/Practicing DS/pds2021-opioids-pds6/10_modified_data/opi_merge_final.parquet", engine='fastparquet')

# vital deaths
vital = pd.read_parquet("/Users/preetkhowaja/Documents/MIDS Fall 2021/Practicing DS/pds2021-opioids-pds6/10_modified_data/Vital.parquet", engine='fastparquet')

# population
population = pd.read_parquet("/Users/preetkhowaja/Documents/MIDS Fall 2021/Practicing DS/pds2021-opioids-pds6/10_modified_data/pop_final.parquet", engine='fastparquet')

In [3]:
opioid.sample(10)

,county_name,state_abbr,fips,BUYER_STATE,BUYER_COUNTY,year,MME
index,,,,,,,
1713,FULTON,GA,13121,GA,FULTON,2011,2.669609e+08
4856,SAN MIGUEL,NM,35047,NM,SAN MIGUEL,2014,1.269468e+07
1009,OKALOOSA,FL,12091,FL,OKALOOSA,2010,6.146017e+07
3380,ST. TAMMANY,LA,22103,LA,ST. TAMMANY,2008,1.275210e+08
1014,OKEECHOBEE,FL,12093,FL,OKEECHOBEE,2006,2.062854e+07
8700,WALLA WALLA,WA,53071,WA,WALLA WALLA,2006,1.624662e+07
2371,TIFT,GA,13277,GA,TIFT,2007,9.289768e+06
278,GENEVA,AL,1061,AL,GENEVA,2014,1.037765e+07
4046,SCOTT,MS,28123,MS,SCOTT,2008,5.824541e+06


In [4]:
vital.sample(10)

,County,County Code,Year,Deaths
3791,"Lake County, IN",18089,2003,29
4143,"Lubbock County, TX",48303,2011,21
4146,"Lubbock County, TX",48303,2014,27
1173,"Charles County, MD",24017,2012,11
899,"Cabell County, WV",54011,2011,37
1970,"District of Columbia, DC",11001,2009,56
2154,"El Paso County, TX",48141,2004,26
4191,"Macomb County, MI",26099,2014,268
5581,"Pitt County, NC",37147,2009,16
1015,"Campbell County, KY",21037,2015,46


In [5]:
population.sample(10)

,STATE,COUNTY,STNAME,CTYNAME,Year,FIPS
32577,17,117,Illinois,Macoupin County,2013,46680
4699,29,1,Missouri,Adair County,2004,24988
13330,16,5,Idaho,Bannock County,2007,79338
2620,48,111,Texas,Dallam County,2003,6148
2716,48,303,Texas,Lubbock County,2003,253388
11358,32,21,Nevada,Mineral County,2006,4864
6473,2,240,Alaska,Southeast Fairbanks Census Area,2005,6530
40816,47,111,Tennessee,Macon County,2015,23076
21421,41,61,Oregon,Union County,2009,25580
34332,46,41,South Dakota,Dewey County,2013,5586


In [6]:
population[population.duplicated(["FIPS", "Year"], keep=False)==True]

,STATE,COUNTY,STNAME,CTYNAME,Year,FIPS
19,1,37,Alabama,Coosa County,2003,11732
82,2,150,Alaska,Kodiak Island Borough,2003,13239
85,2,180,Alaska,Nome Census Area,2003,9210
92,2,261,Alaska,Valdez-Cordova Census Area,2003,9996
143,5,61,Arkansas,Howard County,2003,14098
...,...,...,...,...,...,...
41273,51,690,Virginia,Martinsville city,2015,13259
41334,54,7,West Virginia,Braxton County,2015,14375
41374,54,87,West Virginia,Roane County,2015,14375
41378,54,95,West Virginia,Tyler County,2015,8958


In [7]:
# Merge opioid and vital first

merge1 = pd.merge(opioid, vital, left_on=["fips", "year"], right_on=["County Code", "Year"], how='inner')

merge1

,county_name,state_abbr,fips,BUYER_STATE,BUYER_COUNTY,year,MME,County,County Code,Year,Deaths
0,BALDWIN,AL,1003,AL,BALDWIN,2006,5.103983e+07,"Baldwin County, AL",1003,2006,11
1,BALDWIN,AL,1003,AL,BALDWIN,2007,6.032305e+07,"Baldwin County, AL",1003,2007,24
2,BALDWIN,AL,1003,AL,BALDWIN,2008,6.682852e+07,"Baldwin County, AL",1003,2008,19
3,BALDWIN,AL,1003,AL,BALDWIN,2009,7.638538e+07,"Baldwin County, AL",1003,2009,29
4,BALDWIN,AL,1003,AL,BALDWIN,2010,9.272014e+07,"Baldwin County, AL",1003,2010,24
...,...,...,...,...,...,...,...,...,...,...,...
1775,YAKIMA,WA,53077,WA,YAKIMA,2010,6.746887e+07,"Yakima County, WA",53077,2010,24
1776,YAKIMA,WA,53077,WA,YAKIMA,2011,7.600735e+07,"Yakima County, WA",53077,2011,14
1777,YAKIMA,WA,53077,WA,YAKIMA,2012,8.023265e+07,"Yakima County, WA",53077,2012,25
1778,YAKIMA,WA,53077,WA,YAKIMA,2013,8.391631e+07,"Yakima County, WA",53077,2013,14


In [8]:
# Convert the fips column in merge1 data to strings and fill them to 5 digits. (for better merging with population data)

merge1["fips"] = merge1["fips"].astype(str)

merge1["fips"] = merge1["fips"].apply(lambda x: x.zfill(5))

merge1 = merge1.drop(columns=["year", "County Code"])

# Convert the year column in population data to int64 so that the type matches

population["Year"] = population["Year"].astype(int)

In [9]:
# Merge opioid and vital deaths data with population data, then

merge2 = pd.merge(merge1, population, left_on=["fips", "Year"], right_on=["FIPS", "Year"], how='inner')

merge_final = merge2[["BUYER_STATE", "BUYER_COUNTY", "Year", "MME", "Deaths", "Population", "fips"]]

merge_final["BUYER_STATE"].value_counts()


ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [10]:
## Checking if we have same number of rows for all years. 
merge_final['Year'].value_counts()



NameError: name 'merge_final' is not defined